In [35]:
import sys, os, pathlib


import numpy as np
import pandas as pd
import nibabel as nib
import meshio
import torch

import matplotlib as mpl
import pyvista as pv
import lung_project_git.project as project


print(torch.cuda.is_available())
print(os.getcwd())

False
/projectnb/batmanlab/haozhe/Lung_Mesh_Segment/copd_seg


In [36]:
data_root_scc = "/projectnb/batmanlab/mragoza/COPDGene"

In [18]:
"""ds = project.datasets.copdgene.COPDGeneDataset(
    data_root=data_root_scc
)
ex = next(ds.examples(
    subjects=['16514P'],
    visit='Phase-1',
    variant='TEST',
    state_pairs=[('EXP', 'INSP')]
))"""


In [32]:
!ls -lh /projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW
!find /projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW -maxdepth 1 -name '*.nii.gz' -print
!tree -L 3 /projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1
!ls -lh /projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/images

total 289M
-rw-r--r-- 1 mragoza batmanlab 172M Nov  4 14:27 16514P_EXP_STD_TEM_COPD.nii.gz
-rw-r--r-- 1 mragoza batmanlab 118M Nov  4 14:27 16514P_INSP_STD_TEM_COPD.nii.gz
/projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW/16514P_EXP_STD_TEM_COPD.nii.gz
/projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW/16514P_INSP_STD_TEM_COPD.nii.gz
/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1
├── disps
│   └── 16514P_EXP_STD_TEM_COPD
│       └── EXP_to_INSP.nii.gz
├── images
│   ├── 16514P_EXP_STD_TEM_COPD.nii.gz
│   └── 16514P_INSP_STD_TEM_COPD.nii.gz
├── masks
│   └── 16514P_EXP_STD_TEM_COPD
│       ├── lung_combined.nii.gz
│       ├── lung_lower_lobe_left.nii.gz
│       ├── lung_lower_lobe_right.nii.gz
│       ├── lung_middle_lobe_right.nii.gz
│       ├── lung_regions.nii.gz
│       ├── lung_trachea_bronchia.nii.gz
│       ├── lung_upper_lobe_left.nii.gz
│       ├── lung_upper_lobe_right.nii.gz
│       └── lung_vessels.nii.gz
└── meshes
    └── 16514P_EXP_ST

In [38]:
ds = project.datasets.copdgene.COPDGeneDataset(
    data_root=data_root_scc
)
ex = next(ds.examples(
    subjects=['16514P'],
    visit='Phase-1',
    variant='TEST',            # pulls TEST/images
    state_pairs=[('EXP', 'INSP')],
))
for key in ['fixed_image','fixed_mask','fixed_mesh','disp_field']:
    print(key, ex.paths[key], ex.paths[key].exists())

fixed_image /projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/images/16514P_EXP_STD_TEM_COPD.nii.gz True


KeyError: 'fixed_mask'

In [19]:
ex.paths

{'source_ref': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW/16514P_EXP_STD_TEM_COPD.nii.gz'),
 'source_fixed': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW/16514P_EXP_STD_TEM_COPD.nii.gz'),
 'source_moving': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/Images/16514P/Phase-1/RAW/16514P_INSP_STD_TEM_COPD.nii.gz'),
 'fixed_image': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/images/16514P_EXP_STD_TEM_COPD.nii.gz'),
 'moving_image': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/images/16514P_INSP_STD_TEM_COPD.nii.gz'),
 'binary_mask': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/masks/16514P_EXP_STD_TEM_COPD/lung_combined.nii.gz'),
 'region_mask': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/masks/16514P_EXP_STD_TEM_COPD/lung_regions.nii.gz'),
 'surface_mesh': PosixPath('/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/meshe

In [24]:
from pathlib import Path

mask_p = Path("/projectnb/batmanlab/mragoza/COPDGene/TEST/16514P/Phase-1/masks/16514P_EXP_STD_TEM_COPD/lung_regions.nii.gz")
ex.paths.update({"fixed_mask": mask_p})



In [25]:
def print_tensor_info(*args, **kwargs):
    for i, v in enumerate(args):
        print(f'{i}: {v.shape} {v.dtype} {v.device}')
    for k, v in kwargs.items():
        print(f'{k}: {v.shape} {v.dtype} {v.device}')

device = 'cuda'
dtype = torch.float32

image_nifti = nib.load(ex.paths['fixed_image'])
mask_nifti  = nib.load(ex.paths['fixed_mask'])
disp_nifti  = nib.load(ex.paths['disp_field'])

mesh_io = meshio.read(ex.paths['fixed_mesh'])

affine_array = image_nifti.affine
image_array  = image_nifti.get_fdata()
mask_array   = mask_nifti.get_fdata()
disp_array   = disp_nifti.get_fdata()

image_tensor = torch.as_tensor(image_array, dtype=dtype, device=device).unsqueeze(0)
mask_tensor  = torch.as_tensor(mask_array, dtype=dtype, device=device).unsqueeze(0)
disp_tensor  = torch.as_tensor(disp_array, dtype=dtype, device=device).permute(3,0,1,2)

print_tensor_info(image_tensor, mask_tensor, disp_tensor)

KeyError: 'fixed_mesh'